In [4]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm, trange

In [2]:
base_dir = "/home/aritra/project/quartLT23/data/tiny_imagenet/tiny-imagenet-200"
file = open(f"{base_dir}/wnids.txt", "r")
classes = file.readlines()
# print((classes))
file.close()

classinfo = {}
for i in range(200):
    classinfo[i] = classes[i][:-1]
print(classinfo)  

train_dir = base_dir+"/train"
test_dir = base_dir+"/test"
val_dir = base_dir+"/val"  

del classes

FileNotFoundError: [Errno 2] No such file or directory: '/home/aritra/project/quartLT23/data/tiny_imagenet/tiny-imagenet-200/wnids.txt'

In [ ]:
!unzip tiny-imagenet-200.zip

In [1]:
# TEST CELL



NameError: name 'os' is not defined

In [ ]:
# correcting grayscales

badcount = 0
for i in range(200):
    for j in trange(500):
        img = plt.imread(f"{train_dir}/{classinfo[i]}/images/{classinfo[i]}_{j}.JPEG")
        if img.shape == (64,64):
            badcount += 1
            plt.imsave(f"{train_dir}/{classinfo[i]}/images/{classinfo[i]}_{j}.JPEG",np.append(img,np.append(img,img)).reshape((3, 64, 64)).transpose(1, 2, 0))
            
print(f"grayscale:{badcount}")

# grayscale_train: 1821
# grayscale_val: 168

In [ ]:
for i in range(200):
    train_class = np.array([np.insert(np.array(plt.imread(f"{train_dir}/{classinfo[i]}/images/{classinfo[i]}_0.JPEG")).flatten(),0,i)])
    for j in trange(1,500):
        # print(f"{i}th class, {j}th image)
        train_class = np.append(train_class,[np.insert(np.array(plt.imread(f"{train_dir}/{classinfo[i]}/images/{classinfo[i]}_{j}.JPEG")).flatten(),0,i)],axis=0)
    if i == 0:
        train = np.copy(train_class)
        continue
    train = np.append(train,train_class,axis=0)

In [ ]:
print(train)

np.random.seed(69)
np.random.shuffle(train)
print(train)
np.save("train_shuffled.npy",train)

del train
# grayscale:1821

In [ ]:
#ORganizing test labels
file = open(f"{val_dir}/val_annotations.txt", "r")
labels = file.readlines()
# print((classes))
file.close()

labels2 = {}
for i in range(10000):
    x,y = labels[i].split()[:2]
    for j in range(200):
        if classinfo[j] == y:y = j
    labels2[x] = y
print(labels2)  
# print(classinfo[163])
 
del labels

In [ ]:
test = np.array([])
img = np.array(plt.imread(f"{val_dir}/images/val_0.JPEG")).flatten()
img = np.insert(img,0,labels2[f'val_0.JPEG'])
test = np.array([img])
# print(test)
for j in tqdm(range(10000)):
    if j == 0: continue
    # print(f"{j}th image")
    test = np.append(test,[np.insert(np.array(plt.imread(f"{val_dir}/images/val_{j}.JPEG")).flatten(),0,labels2[f'val_{j}.JPEG'])],axis=0)

In [ ]:
# np.random.seed(69)
# np.random.shuffle(test)
# print(test)
np.save("test.npy",test)

del test
# grayscale:168

In [ ]:
!rm -rf tiny-imagenet-200